In [1]:
import pdfplumber
from llm_guard.input_scanners import PromptInjection, InvisibleText
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llm_guard.input_scanners.prompt_injection import MatchType
import pytesseract
from docx import Document
from PIL import Image
from langchain_core.documents import Document as LCDocument
import torch
import pandas as pd
import os
import ast
import json
import re
import time
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from deepeval.test_case import LLMTestCase
from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.metrics import (
    FaithfulnessMetric,
    AnswerRelevancyMetric,
    ContextualRecallMetric,
    ContextualPrecisionMetric,
    ContextualRelevancyMetric
)

from openai import OpenAI
from qdrant_client import QdrantClient, models
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_core.embeddings import Embeddings
from typing import List
import requests
import uuid
import gc
import tqdm

In [19]:
file_list = ["1.pdf", "2.pdf", "3.pdf", "4.png", "5.docx"]
file_path = "HW"
EMBEDDING_API_URL = "http://ws-04.wade0426.me/embed"
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "rag_homework_day7_api"
LLM_BASE_URL = "https://ws-06.huannago.com/v1"
RERANKER_MODEL_PATH = os.path.expanduser("../day6/Qwen3-Reranker-0.6B")
LLM_API_KEY = "day6hw"
LLM_MODEL_NAME = "gemma-3-27b-it"
PREDICT_INPUT = "HW/questions.csv"
PREDICT_OUPUT = "HW/output.csv"

In [3]:
file_list = ["1.pdf", "2.pdf", "3.pdf", "4.png", "5.docx"]
file_path = "HW"
docs_content = {}

print("--- 開始 IDP 處理 ---")

for file_name in file_list:
    full_path = os.path.join(file_path, file_name)

    if not os.path.exists(full_path):
        print(f"* 找不到檔案: {file_name}")
        continue

    print(f"正在處理: {file_name}...")
    extracted_text = ""

    # ==========================================
    # 1. 針對 PDF (包含 3.pdf 的表格處理)
    # ==========================================
    if file_name.endswith(".pdf"):
        try:
            with pdfplumber.open(full_path) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        extracted_text += text + "\n"

                    tables = page.extract_tables()
                    for table in tables:
                        table_str = ""
                        for row in table:
                            # 清洗 row 中的 None，轉為字串
                            cleaned_row = [str(cell) if cell is not None else "" for cell in row]
                            table_str += " | ".join(cleaned_row) + "\n"

                        extracted_text += "\n[表格內容開始]\n" + table_str + "\n[表格內容結束]\n"

                print(f"✅ [{file_name}] PDF (含表格) 提取成功")
        except Exception as e:
            print(f"❌ PDF 讀取錯誤 {file_name}: {e}")

    # ==========================================
    # 2. 針對 PNG (針對 4.png 的 OCR 處理)
    # ==========================================
    elif file_name.endswith(".png"):
        try:
            image = Image.open(full_path)
            ocr_text = pytesseract.image_to_string(image, lang='chi_tra+eng')

            extracted_text = ocr_text
            print(f"✅ [{file_name}] OCR 辨識成功")
        except Exception as e:
            print(f"❌ OCR 失敗 {file_name}: {e}")
            print("提示: 請確認是否已安裝 Tesseract 軟體並設定環境變數")

    # ==========================================
    # 3. 針對 DOCX (5.docx)
    # ==========================================
    elif file_name.endswith(".docx"):
        try:
            doc = Document(full_path)
            for para in doc.paragraphs:
                extracted_text += para.text + "\n"
            print(f"✅ [{file_name}] Word 讀取成功")
        except Exception as e:
            print(f"❌ Word 讀取錯誤 {file_name}: {e}")

    # --- 存入結果 ---
    if extracted_text.strip():
        docs_content[file_name] = extracted_text
    else:
        print(f"⚠️ 警告: {file_name} 沒有提取到任何文字")

print("-" * 30)

✅ [4.png] OCR 辨識成功
正在處理: 5.docx...
✅ [5.docx] Word 讀取成功
------------------------------


In [4]:
# 1. 初始化切分器
text_splitter = RecursiveCharacterTextSplitter(
    # 分隔符優先順序：雙換行(段落) > 單換行(行) > 空格 > 空字串
    separators=["\n\n", "\n", " ", ""],
    chunk_size=500,    # 每個區塊大約 500 個字元
    chunk_overlap=50,  # 滑動視窗：保留 50 個字元的重疊，避免語意斷裂
    length_function=len,
    is_separator_regex=False,
)

# 2. 準備將你的字典轉換成 LangChain 的 Document 物件
all_documents = []

for filename, text in docs_content.items():
    # 建立 Document 物件，記得把檔名放入 metadata，這是之後 RAG 引用來源(Source)的關鍵
    doc = LCDocument(
        page_content=text,
        metadata={"source": filename}
    )
    all_documents.append(doc)

# 3. 執行切分
split_docs = text_splitter.split_documents(all_documents)

In [5]:
scanner = PromptInjection(threshold=0.5, match_type=MatchType.FULL)
for filename, text_content in docs_content.items():
    print(f"正在掃描 {filename} ...")

    # 確保內容是字串，避免錯誤
    if not isinstance(text_content, str):
        text_content = str(text_content)

    # 執行掃描
    sanitized_text, is_valid, score = scanner.scan(text_content)

    if not is_valid:
        print(f"*  警告：在 [{filename}] 發現隱藏文字！(Score: {score})")
    else:
        print(f"OK：[{filename}] 未發現隱藏文字。")

OK：[5.docx] 未發現隱藏文字。


In [7]:
if torch.cuda.is_available():
    device_obj = torch.device("cuda")
elif torch.backends.mps.is_available():
    device_obj = torch.device("mps")
else:
    device_obj = torch.device("cpu")

print(f"⏳ 使用裝置: {device_obj}")

⏳ 使用裝置: mps


In [8]:
class CustomAPIEmbeddings(Embeddings):
    def __init__(self, api_url):
        self.api_url = api_url

    def _call_api(self, texts: List[str]) -> List[List[float]]:
        data = {
            "texts": texts,
            "normalize": True,
            "batch_size": 32
        }

        try:
            response = requests.post(self.api_url, json=data, timeout=60)
            if response.status_code == 200:
                result = response.json()
                return result.get('embeddings', [])
            else:
                print(f"❌ API Error Code: {response.status_code}")
                return []
        except Exception as e:
            print(f"❌ API Exception: {e}")
            return []

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self._call_api(texts)

    def embed_query(self, text: str) -> List[float]:
        results = self._call_api([text])
        if results and len(results) > 0:
            return results[0]
        return []


print(f"* 初始化 Embedding API ({EMBEDDING_API_URL})...")
embedding_model = CustomAPIEmbeddings(EMBEDDING_API_URL)
try:
    print("* 測試 Embedding API 連線...")
    test_vec = embedding_model.embed_query("測試")
    if test_vec:
        print(f"✅ API 連線成功！向量維度: {len(test_vec)}")
    else:
        print("❌ API 連線失敗，回傳為空")
        exit()
except Exception as e:
    print(f"❌ API 測試發生例外錯誤: {e}")
    exit()

✅ API 連線成功！向量維度: 4096


In [9]:
class SimpleLLMClient:
    def __init__(self, base_url, model_name, api_key):
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.model_name = model_name

    def generate(self, prompt: str) -> str:
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"LLM Error: {e}")
            return "Error generating response."


print("⏳ 初始化 LLM Client...")
llm_client = SimpleLLMClient(LLM_BASE_URL, LLM_MODEL_NAME, LLM_API_KEY)


⏳ 初始化 LLM Client...


In [10]:
client = QdrantClient(url=QDRANT_URL)

In [11]:
def init_qdrant_collection(documents):
    """
    建立 Qdrant Collection 並寫入已切分的資料
    Args:
        documents: List[Document] (由 LangChain 切分好的文件列表)
    """

    # 1. 檢查資料是否為空
    if not documents:
        print("⚠️ 傳入的文件列表為空，跳過寫入。")
        return

    print(f"🔄 正在重置集合 {COLLECTION_NAME}...")
    try:
        client.delete_collection(COLLECTION_NAME)
        print(f"* 已刪除舊集合 {COLLECTION_NAME}")
    except Exception:
        pass

    # 2. 建立新的 Collection
    # 注意: size=4096 必須與你使用的 Embedding 模型維度一致
    # (例如 OpenAI text-embedding-3-large 是 3072, text-embedding-ada-002 是 1536)
    print(f"⏳ 建立新 Collection: {COLLECTION_NAME}...")
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config={
            "dense": models.VectorParams(
                distance=models.Distance.COSINE,
                size=1536, # <--- 請確認這裡是否符合你的 Embedding 模型維度！
            ),
        },
        # 啟用混合檢索 (Hybrid Search) 的稀疏向量配置
        sparse_vectors_config={
            "sparse": models.SparseVectorParams(modifier=models.Modifier.IDF)
        },
    )

    print(f"📊 準備處理 {len(documents)} 個 Chunks...")

    # 3. 準備純文字列表以進行 Embedding
    # document 物件不能直接 embed，要取出 page_content
    texts_to_embed = [doc.page_content for doc in documents]

    print("⏳ 計算 Embeddings (Dense Vectors)...")
    try:
        doc_embeddings = embedding_model.embed_documents(texts_to_embed)
    except Exception as e:
        print(f"❌ Embedding 計算失敗: {e}")
        return

    if len(doc_embeddings) != len(documents):
        print("❌ Embedding 數量與文件數量不符！")
        return

    # 4. 準備寫入 Qdrant 的 Points
    points = []
    print("⏳ 正在封裝資料...")

    for doc, embedding in zip(documents, doc_embeddings):
        # 產生 UUID
        point_id = uuid.uuid4().hex

        # 建立 Payload (包含文字與來源 metadata)
        payload = {
            "text": doc.page_content,
            "source": doc.metadata.get("source", "unknown"), # 把來源檔名存進去
            "page": doc.metadata.get("page", 0) # 如果有的話
        }

        # 建立 Point
        point = models.PointStruct(
            id=point_id,
            vector={
                "dense": embedding,
                # 這裡使用 Qdrant 的各種 BM25 支援方式，傳入純文字
                "sparse": models.Document(text=doc.page_content, model="Qdrant/bm25"),
            },
            payload=payload,
        )
        points.append(point)

    # 5. 批次寫入 (Upsert)
    batch_size = 50
    print(f"⏳ 開始寫入 {len(points)} 筆資料至 Qdrant...")

    for i in tqdm(range(0, len(points), batch_size), desc="Upserting"):
        batch_points = points[i : i + batch_size]
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=batch_points
        )

    print(f"✅ 資料寫入完成！Collection: {COLLECTION_NAME}")

In [12]:
print("* 載入 Reranker 模型...")
reranker_tokenizer = AutoTokenizer.from_pretrained(
    RERANKER_MODEL_PATH, local_files_only=True, trust_remote_code=True
)
reranker_model = AutoModelForCausalLM.from_pretrained(
    RERANKER_MODEL_PATH, local_files_only=True, trust_remote_code=True
).to(device_obj).eval()

token_false_id = reranker_tokenizer.convert_tokens_to_ids("no")
token_true_id = reranker_tokenizer.convert_tokens_to_ids("yes")
max_reranker_length = 8192

prefix = "<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n<|im_start|>user\n"
suffix = "<|im_end|>\n<|im_start|>assistant\n"
prefix_tokens = reranker_tokenizer.encode(prefix, add_special_tokens=False)
suffix_tokens = reranker_tokenizer.encode(suffix, add_special_tokens=False)

⏳ 載入 Reranker 模型...


In [13]:
def compute_rerank_scores(pairs, batch_size=4):
    """
    分批計算 Reranker 分數，避免 MPS Out Of Memory
    batch_size: 建議設小一點 (例如 2 或 4)
    """
    all_scores = []

    # 使用 tqdm 顯示 Rerank 進度 (可選)
    # for i in range(0, len(pairs), batch_size):

    for i in range(0, len(pairs), batch_size):
        batch_pairs = pairs[i: i + batch_size]

        processed_inputs = []
        for pair in batch_pairs:
            pair_ids = reranker_tokenizer.encode(
                pair, add_special_tokens=False, truncation=True,
                max_length=max_reranker_length - len(prefix_tokens) - len(suffix_tokens)
            )
            full_ids = prefix_tokens + pair_ids + suffix_tokens
            processed_inputs.append(reranker_tokenizer.decode(full_ids))

        inputs = reranker_tokenizer(
            processed_inputs, padding=True, truncation=True, return_tensors="pt", max_length=max_reranker_length
        )

        # 移動到 GPU
        for key in inputs:
            inputs[key] = inputs[key].to(device_obj)

        with torch.no_grad():
            logits = reranker_model(**inputs).logits[:, -1, :]
            scores = logits[:, token_true_id].exp().tolist()
            all_scores.extend(scores)

        # 清理 GPU 記憶體
        del inputs, logits, scores
        if device_obj.type == "mps":
            torch.mps.empty_cache()
        elif device_obj.type == "cuda":
            torch.cuda.empty_cache()
        gc.collect()

    return all_scores

In [14]:
def rerank_documents(query, documents):
    if not documents: return []

    formatted_pairs = [
        f"<Instruct>: 根據查詢檢索相關文件\n<Query>: {query}\n<Document>: {doc}"
        for doc in documents
    ]

    # 這裡呼叫修改後的函數，batch_size 設為 4 (若還是爆記憶體，請改成 2 或 1)
    scores = compute_rerank_scores(formatted_pairs, batch_size=4)

    doc_scores = list(zip(documents, scores))
    doc_scores.sort(key=lambda x: x[1], reverse=True)
    return doc_scores

In [15]:
def query_rewrite(query: str) -> str:
    prompt = f"""
    你是一個搜尋引擎優化專家。請將以下使用者的問題改寫為更精確、適合做語義檢索的關鍵字查詢。
    保留核心意圖，去除贅詞，並針對自來水公司相關業務進行優化。
    只輸出改寫後的句子，不要有任何解釋。

    使用者問題: {query}
    改寫後查詢:
    """
    rewritten = llm_client.generate(prompt).strip()
    return rewritten

In [16]:
def hybrid_search_with_rerank(query: str, initial_limit=20, final_limit=3):
    query_vec = embedding_model.embed_query(query)

    try:
        response = client.query_points(
            collection_name=COLLECTION_NAME,
            prefetch=[
                models.Prefetch(
                    query=models.Document(text=query, model="Qdrant/bm25"),
                    using="sparse",
                    limit=initial_limit,
                ),
                models.Prefetch(
                    query=query_vec,
                    using="dense",
                    limit=initial_limit,
                ),
            ],
            query=models.FusionQuery(fusion=models.Fusion.RRF),
            limit=initial_limit,
        )
        candidate_docs = [point.payload["text"] for point in response.points]
    except Exception as e:
        print(f"Search Error: {e}")
        return []

    if not candidate_docs:
        return []

    # 這裡進行 Rerank
    top_results = rerank_documents(query, candidate_docs)[:final_limit]
    return top_results

In [17]:
def main():
    print(f"📂 讀取 Excel: {PREDICT_INPUT}")
    if not os.path.exists(PREDICT_INPUT):
        print("❌ 檔案不存在")
        return
    df =  pd.read_csv(PREDICT_INPUT)

    if 'q_id' not in df.columns: df['q_id'] = None
    if 'answer' not in df.columns: df['answer'] = None

    df['q_id'] = df['q_id'].astype('object')
    df['answer'] = df['answer'].astype('object')

    # 【新增】用來暫存 Ground Truth (Context) 的列表
    ground_truth_list = []

    print("🚀 開始處理問題...")

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        original_question = str(row['questions'])

        current_uuid = str(uuid.uuid4())
        refined_query = query_rewrite(original_question)
        search_results = hybrid_search_with_rerank(refined_query)
        retrieval_context = [doc for doc, score in search_results]
        context_str = "\n".join(retrieval_context)

        ground_truth_list.append({
                    "id": current_uuid,
                    "questions": original_question,
                    "contexts": retrieval_context,  # DeepEval 需要 list
                    "ground_truth": ""  # 預留欄位，DeepEval 的 Recall 需要這個 (標準答案)
                })

        qa_prompt = f"""
        你是一個專業的工廠資訊部助手。請根據【參考資料】回答使用者的【問題】。

        規範：
        1. 答案必須基於參考資料，不要編造。
        2. 如果參考資料不足以回答，請回答「目前資訊不足，建議聯繫客服」。
        3. 語氣親切、專業。

        【參考資料】：
        {context_str}

        【問題】：{original_question}

        【回答】：
        """
        answer = llm_client.generate(qa_prompt)

        # 4. 將 Answer 寫回原本的 DataFrame
        df.at[index, 'id'] = current_uuid
        df.at[index, 'answer'] = answer

    df.to_excel(PREDICT_OUPUT, index=False)
    print(f"✅ Answer 處理完成！結果已儲存至: {PREDICT_OUPUT}")

    # 檔案 2：存 Ground Truth (Context) 的 CSV
    gt_df = pd.DataFrame(ground_truth_list)

    # 存成 CSV (建議用 utf-8-sig 以免中文亂碼)
    gt_df.to_csv("ground_truth.csv", index=False, encoding='utf-8-sig')
    print(f"✅ Ground Truth (Context) 已儲存至: ground_truth.csv")

In [20]:
if __name__ == "__main__":
    main()

KeyboardInterrupt: 